<a href="https://colab.research.google.com/github/yuziny/Data-Analysis/blob/main/2_%EA%B5%AC%EB%B3%84_%EC%9D%B4%EB%8F%99%EC%9D%B8%EA%B5%AC_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울시 생활정보 기반 대중교통 수요 분석

버스 노선 추가가 필요한 서울시 내 자치구 선정

2. 구 별 이동인구 분석

In [ ]:
!pip install pandas seaborn

데이터

- 2.seoul_moving.csv
- 서울시 구별 이동 2022년 4월 데이터
- 도착시간: 0 -> "0:00~0:59"
- 출발 자치구: 자치구 코드
- 성별: F(여),M(남)
- 나이: 10 ~ 79세(5세 단위), 0~10세, 80세 이상으로 구분
- 유형: H(야간상주지), W(주간상주지), E(기타)
- 평균 이동 시간: 동일 열 내 이동인구의 월 단위 평균 이동시간(10분 단위)
- 이동인구: 추정 합산값으로 소수점 표출, 개인정보 비식별화를 위해 3명 미만(0~3)의 경우 * 처리


In [ ]:
import pandas as pd
seoul_moving = pd.read_csv('/content/drive/MyDrive/포트폴리오/2. seoul_moving.csv', sep=',', encoding='utf-8', index_col=0)
seoul_moving.shape

(28923054, 10)

기본 정보 확인

In [ ]:
seoul_moving.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28923054 entries, 0 to 968462
Data columns (total 10 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   대상연월         int64 
 1   요일           object
 2   도착시간         int64 
 3   출발 시군구 코드    int64 
 4   도착 시군구 코드    int64 
 5   성별           object
 6   나이           int64 
 7   이동유형         object
 8   평균 이동 시간(분)  int64 
 9   이동인구(합)      object
dtypes: int64(6), object(4)
memory usage: 2.4+ GB


In [ ]:
seoul_moving.describe()

,대상연월,도착시간,출발 시군구 코드,도착 시군구 코드,나이,평균 이동 시간(분)
count,28923054.0,2.892305e+07,2.892305e+07,2.892305e+07,2.892305e+07,2.892305e+07
mean,202204.0,1.365597e+01,1.712362e+04,1.719750e+04,4.279614e+01,3.279829e+01
std,0.0,5.849675e+00,9.046478e+03,9.063818e+03,1.777485e+01,2.584213e+01
min,202204.0,0.000000e+00,1.101000e+04,1.101000e+04,0.000000e+00,1.000000e+01
25%,202204.0,1.000000e+01,1.110000e+04,1.111000e+04,3.000000e+01,2.000000e+01
50%,202204.0,1.400000e+01,1.120000e+04,1.120000e+04,4.500000e+01,3.000000e+01
75%,202204.0,1.800000e+01,3.101400e+04,3.102100e+04,5.500000e+01,4.000000e+01
max,202204.0,2.300000e+01,3.900000e+04,3.900000e+04,8.000000e+01,7.700000e+02


In [ ]:
seoul_moving.head()

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202204,일,0,11010,11010,F,0,EE,10,28.02
1,202204,일,0,11010,11010,F,0,EH,10,27.52
2,202204,일,0,11010,11010,F,0,HE,10,27.83
3,202204,일,0,11010,11010,F,10,HW,10,6.05
4,202204,일,0,11010,11010,F,10,EH,90,5.95


2022년 4월 일요일에 11010(종로구)에서 11010(종로구)으로 이동하여 0시~ 0시59분 사이에 도착, E(기타)에서 H(야간상주지)로 이동한 0~ 10세 여성은 총 27.72명이며 이들의 평균 이동시간은 10분이다.

결측치 처리(NaN이 아닌)

In [ ]:
# 이동인구(합)은 숫자가 아닌 object이다

In [ ]:
seoul_moving['이동인구(합)'].value_counts()

*          3842041
3.3         399374
6.05        366830
3.29        289731
3.31        232069
            ...   
2352.37          1
1617.99          1
3141.86          1
1485.54          1
979.87           1
Name: 이동인구(합), Length: 135812, dtype: int64

In [ ]:
# * 로 표현된 값 대체
# 최빈값, 평균값도 알 수 없으므로 0 ~ 3의 중위값인 1.5로 대체
seoul_moving['이동인구(합)'].replace('*', 1.5, inplace=True)

In [ ]:
seoul_moving['이동인구(합)'].value_counts()

1.5        3842041
3.3         399374
6.05        366830
3.29        289731
3.31        232069
            ...   
2352.37          1
1617.99          1
3141.86          1
1485.54          1
979.87           1
Name: 이동인구(합), Length: 135812, dtype: int64

In [ ]:
seoul_moving['이동인구(합)'] = seoul_moving['이동인구(합)'].astype(float)

In [ ]:
seoul_moving.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28923054 entries, 0 to 968462
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   대상연월         int64  
 1   요일           object 
 2   도착시간         int64  
 3   출발 시군구 코드    int64  
 4   도착 시군구 코드    int64  
 5   성별           object 
 6   나이           int64  
 7   이동유형         object 
 8   평균 이동 시간(분)  int64  
 9   이동인구(합)      float64
dtypes: float64(1), int64(6), object(3)
memory usage: 2.4+ GB


이동유형, 요일, 성별 변수의 value 수 파악

In [ ]:
# H(야간상주지), W(주간상주지), E(기타)
seoul_moving['이동유형'].value_counts()

EH    5757073
HE    5520005
EE    5085099
WH    3108026
WE    2809148
HW    2695189
EW    2304535
WW    1026134
HH     617845
Name: 이동유형, dtype: int64

In [ ]:
# 요일 별
seoul_moving['요일'].value_counts()

토    4690806
금    4641881
목    4011291
수    3955646
화    3926876
월    3906272
일    3790282
Name: 요일, dtype: int64

2022년 4월의 요일 수
- 일:4, 월:4, 화:4, 목:4, 금:5, 토:5

심야 이동이 가장 많은 요일 확인

In [ ]:
# 심야시간은 0:00 ~ 4:59
temp = seoul_moving.loc[seoul_moving['도착시간'].isin([0, 1, 2, 3, 4]), ['요일', '도착시간']]
temp.groupby('요일')['도착시간'].count().reset_index(name='심야시간이동건수')

,요일,심야시간이동건수
0,금,359012
1,목,308651
2,수,303357
3,월,273062
4,일,331392
5,토,423995
6,화,292786


In [ ]:
seoul_moving.loc[seoul_moving['도착시간'].between(0,4)][['요일']].value_counts()

요일
토     423995
금     359012
일     331392
목     308651
수     303357
화     292786
월     273062
dtype: int64

서울 지역의 데이터만 추출

자치구 코드 정보

https://data.seoul.go.kr/dataVisual/seoul/seoulLivingMigration.do

In [ ]:
seoul_moving

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202204,일,0,11010,11010,F,0,EE,10,28.02
1,202204,일,0,11010,11010,F,0,EH,10,27.52
2,202204,일,0,11010,11010,F,0,HE,10,27.83
3,202204,일,0,11010,11010,F,10,HW,10,6.05
4,202204,일,0,11010,11010,F,10,EH,90,5.95
...,...,...,...,...,...,...,...,...,...,...
968458,202204,토,23,39000,11250,M,45,EH,50,3.16
968459,202204,토,23,39000,11250,M,50,EH,70,3.27
968460,202204,토,23,39000,11250,M,55,EE,50,3.31
968461,202204,토,23,39000,11250,M,55,EH,50,3.28


In [ ]:
# 출발 시군구 코드, 도착 시군구 코드로 출발과 도착이 모두 서울인 데이터
seoul_code = list(range(11010, 11251, 10))
data = seoul_moving.loc[seoul_moving['출발 시군구 코드'].isin(seoul_code) & seoul_moving['도착 시군구 코드'].isin(seoul_code)]
data

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202204,일,0,11010,11010,F,0,EE,10,28.02
1,202204,일,0,11010,11010,F,0,EH,10,27.52
2,202204,일,0,11010,11010,F,0,HE,10,27.83
3,202204,일,0,11010,11010,F,10,HW,10,6.05
4,202204,일,0,11010,11010,F,10,EH,90,5.95
...,...,...,...,...,...,...,...,...,...,...
920410,202204,토,23,11250,11250,M,80,WH,10,85.10
920411,202204,토,23,11250,11250,M,80,EH,10,28.36
920412,202204,토,23,11250,11250,M,80,EE,20,37.99
920413,202204,토,23,11250,11250,M,80,HE,10,9.42


서울지역의 구 별로 groupby

* 서울 구별 코드
    * 11010	종로구
    * 11020	중구
    * 11030	용산구
    * 11040	성동구
    * 11050	광진구
    * 11060	동대문구
    * 11070	중랑구
    * 11080	성북구
    * 11090	강북구
    * 11100	도봉구
    * 11110	노원구
    * 11120	은평구
    * 11130	서대문구
    * 11140	마포구
    * 11150	양천구
    * 11160	강서구
    * 11170	구로구
    * 11180	금천구
    * 11190	영등포구
    * 11200	동작구
    * 11210	관악구
    * 11220	서초구
    * 11230	강남구
    * 11240	송파구
    * 11250	강동구

In [ ]:
temp = data.loc[data['출발 시군구 코드'] != data['도착 시군구 코드']]
temp.groupby('출발 시군구 코드', as_index=False)['도착 시군구 코드'].count().sort_values(by='도착 시군구 코드', ascending=False)

,출발 시군구 코드,도착 시군구 코드
22,11230,526735
1,11020,469050
21,11220,464732
0,11010,457462
18,11190,455065
13,11140,444103
23,11240,422215
3,11040,413894
5,11060,410447
7,11080,408022


In [ ]:
temp.groupby('도착 시군구 코드', as_index=False)['출발 시군구 코드'].count().sort_values(by='출발 시군구 코드',ascending=False)

,도착 시군구 코드,출발 시군구 코드
22,11230,507324
21,11220,454441
18,11190,450079
1,11020,444247
0,11010,436008
13,11140,435965
23,11240,421097
7,11080,414067
5,11060,410776
3,11040,409946


낮 시간만 분석 대상에 넣기 위해 심야버스 제외
- 도착시간 0부터 4까지(0:00 ~ 4:59)

In [ ]:
days = list(range(5, 24))
data = data.loc[data['도착시간'].isin(days)]
data

,대상연월,요일,도착시간,출발 시군구 코드,도착 시군구 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202204,일,5,11010,11010,F,10,HE,10,5.96
1,202204,일,5,11010,11010,F,10,WH,10,5.88
2,202204,일,5,11010,11010,F,15,HE,10,4.06
3,202204,일,5,11010,11010,F,15,EW,10,4.06
4,202204,일,5,11010,11010,F,15,EH,10,10.08
...,...,...,...,...,...,...,...,...,...,...
920410,202204,토,23,11250,11250,M,80,WH,10,85.10
920411,202204,토,23,11250,11250,M,80,EH,10,28.36
920412,202204,토,23,11250,11250,M,80,EE,20,37.99
920413,202204,토,23,11250,11250,M,80,HE,10,9.42


'도착 시군구 코드' 기준으로 각 구별로 '평균 이동 시간(분)', '이동인구(합)'의 평균과 합 도출

In [ ]:
seoul_moving_mean = data.groupby('도착 시군구 코드', as_index=False)[['평균 이동 시간(분)', '이동인구(합)']].mean()
seoul_moving_mean

,도착 시군구 코드,평균 이동 시간(분),이동인구(합)
0,11010,21.349599,44.235978
1,11020,21.012462,44.516072
2,11030,21.364786,36.903883
3,11040,21.740324,42.685596
4,11050,22.474283,46.762548
5,11060,21.673451,39.942708
6,11070,23.412789,41.012753
7,11080,22.160283,45.816273
8,11090,23.811274,39.128470
9,11100,24.489976,37.882283


In [ ]:
seoul_moving_sum = data.groupby('도착 시군구 코드', as_index=False)[['평균 이동 시간(분)', '이동인구(합)']].sum()
seoul_moving_sum

,도착 시군구 코드,평균 이동 시간(분),이동인구(합)
0,11010,9356120,19385709.60
1,11020,9306020,19715322.61
2,11030,8514380,14707082.87
3,11040,8887140,17449273.95
4,11050,8625540,17947278.99
5,11060,8838130,16288077.28
6,11070,7939300,13907465.59
7,11080,9072420,18757182.35
8,11090,7649610,12570412.30
9,11100,6995880,10821566.75


In [ ]:
seoul_moving_mean.rename(columns={'이동인구(합)': '이동인구평균'}, inplace=True)
seoul_moving_sum.rename(columns={'평균 이동 시간(분)': '총 이동시간', '이동인구(합)': '총 이동인구'}, inplace=True)

In [ ]:
merge_moving = pd.concat([seoul_moving_mean, seoul_moving_sum], axis=1)
merge_moving

,도착 시군구 코드,평균 이동 시간(분),이동인구평균,도착 시군구 코드,총 이동시간,총 이동인구
0,11010,21.349599,44.235978,11010,9356120,19385709.60
1,11020,21.012462,44.516072,11020,9306020,19715322.61
2,11030,21.364786,36.903883,11030,8514380,14707082.87
3,11040,21.740324,42.685596,11040,8887140,17449273.95
4,11050,22.474283,46.762548,11050,8625540,17947278.99
5,11060,21.673451,39.942708,11060,8838130,16288077.28
6,11070,23.412789,41.012753,11070,7939300,13907465.59
7,11080,22.160283,45.816273,11080,9072420,18757182.35
8,11090,23.811274,39.128470,11090,7649610,12570412.30
9,11100,24.489976,37.882283,11100,6995880,10821566.75


In [ ]:
merge_moving.to_csv('df_seoul_moving.csv', index=False)

데이터 분석하기

In [ ]:
m = data.groupby('이동유형', as_index=False)['평균 이동 시간(분)'].mean()
m

,이동유형,평균 이동 시간(분)
0,EE,24.109374
1,EH,23.594629
2,EW,23.167498
3,HE,23.629888
4,HH,17.576836
5,HW,22.269933
6,WE,22.919409
7,WH,22.454969
8,WW,20.746173


In [ ]:
r = data.groupby('나이', as_index=False)['평균 이동 시간(분)'].mean()
r

,나이,평균 이동 시간(분)
0,0,10.115274
1,10,15.093064
2,15,19.020634
3,20,27.612203
4,25,30.182256
5,30,27.715819
6,35,24.047294
7,40,22.546316
8,45,22.731846
9,50,23.640662


In [ ]:
d = data.groupby('요일', as_index=False)['이동인구(합)'].mean()
d

,요일,이동인구(합)
0,금,59.135809
1,목,50.334348
2,수,50.424424
3,월,50.816801
4,일,41.132863
5,토,52.294864
6,화,50.750378


1. 나이에 따른 평균 이동 시간은 분포 이룬다
2. 20대가 평균 이동 시간이 가장 길다
3. 금요일은 이동인구가 많다